In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime, date, time
import os
import uuid
from pathlib import Path
import openpyxl
from openpyxl import Workbook

# Configuration de la page
st.set_page_config(
    page_title="Espace Enseignant - STATO-SPHERE PREPAS",
    page_icon="👨‍🏫",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS personnalisé
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(90deg, #2d5a27 0%, #4a8f42 100%);
        padding: 2rem;
        border-radius: 10px;
        margin-bottom: 2rem;
        text-align: center;
        color: white;
    }
    
    .teacher-info {
        background: linear-gradient(135deg, #27ae60 0%, #2ecc71 100%);
        padding: 1.5rem;
        border-radius: 10px;
        color: white;
        margin-bottom: 2rem;
        text-align: center;
    }
    
    .form-container {
        background: white;
        padding: 2rem;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        margin-bottom: 2rem;
    }
    
    .success-box {
        background: #d4edda;
        color: #155724;
        padding: 1rem;
        border-radius: 5px;
        border: 1px solid #c3e6cb;
        margin: 1rem 0;
    }
    
    .error-box {
        background: #f8d7da;
        color: #721c24;
        padding: 1rem;
        border-radius: 5px;
        border: 1px solid #f5c6cb;
        margin: 1rem 0;
    }
    
    .info-box {
        background: #d1ecf1;
        color: #0c5460;
        padding: 1rem;
        border-radius: 5px;
        border: 1px solid #bee5eb;
        margin: 1rem 0;
    }
    
    .stat-card {
        background: linear-gradient(135deg, #27ae60 0%, #2ecc71 100%);
        padding: 1.5rem;
        border-radius: 10px;
        text-align: center;
        color: white;
        margin-bottom: 1rem;
    }
    
    .login-container {
        background: white;
        padding: 3rem;
        border-radius: 15px;
        box-shadow: 0 8px 16px rgba(0, 0, 0, 0.1);
        max-width: 500px;
        margin: 2rem auto;
    }
    
    .seance-card {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 8px;
        border-left: 4px solid #27ae60;
        margin-bottom: 1rem;
    }
</style>
""", unsafe_allow_html=True)

# Configuration des fichiers Excel
EXCEL_FILE = "stato_sphere_data.xlsx"

# Listes de choix
CONCOURS_CHOICES = ["ISE LONG / AS", "ISE ECO", "ISE MATH", "IFORD B", "IFORD A", "TSS"]
CENTRES_CHOICES = ["Douala", "Yaoundé", "Dschang"]
SEXE_CHOICES = ["Homme", "Femme"]
COURS_CHOICES = [
    "MATHEMATIQUES", "FRANCAIS", "CULTURE GENERALE", "ECONOMIE",
    "ANGLAIS", "STATISTIQUES", "COMPTABILITE", "INFORMATIQUE",
    "DEMOGRAPHIE", "GEOGRAPHIE", "PHYSIQUE", "CHIMIE", "BIOLOGIE"
]

# Base de données des enseignants (à terme, cela pourrait venir d'Excel)
ENSEIGNANTS_DB = {
    "mbang.pierre": {
        "nom": "Dr. MBANG Pierre",
        "password": "mbang123",
        "matieres": ["MATHEMATIQUES", "STATISTIQUES"],
        "centres": ["Yaoundé", "Dschang"]
    },
    "njoya.marie": {
        "nom": "Prof. NJOYA Marie",
        "password": "njoya123",
        "matieres": ["FRANCAIS", "CULTURE GENERALE"],
        "centres": ["Douala", "Yaoundé"]
    },
    "fomba.jean": {
        "nom": "M. FOMBA Jean",
        "password": "fomba123",
        "matieres": ["ECONOMIE", "COMPTABILITE"],
        "centres": ["Dschang"]
    },
    "kamga.sylvie": {
        "nom": "Mme. KAMGA Sylvie",
        "password": "kamga123",
        "matieres": ["ANGLAIS", "CULTURE GENERALE"],
        "centres": ["Douala"]
    },
    "tchoumi.paul": {
        "nom": "Dr. TCHOUMI Paul",
        "password": "tchoumi123",
        "matieres": ["DEMOGRAPHIE", "GEOGRAPHIE"],
        "centres": ["Yaoundé", "Douala"]
    }
}

def init_excel_file():
    """Initialise le fichier Excel avec les feuilles nécessaires"""
    if not os.path.exists(EXCEL_FILE):
        wb = Workbook()
        wb.remove(wb.active)
        
        headers = {
            "Étudiants": ["Matricule", "Nom", "Prénom", "Sexe", "Concours1", "Concours2", "Concours3", 
                         "Téléphone", "DateArrivée", "Etablissement", "Centre"],
            "Séances": ["ID", "Date", "Matière", "HeureArrivée", "HeureDepart", "Classe", "IntituléCours", "Centre", "idEnseignant", "NombreEtudiants"]
        }
        
        for sheet_name, header in headers.items():
            ws = wb.create_sheet(title=sheet_name)
            ws.append(header)
        
        wb.save(EXCEL_FILE)

def save_to_excel(sheet_name, data):
    """Sauvegarde les données dans la feuille Excel spécifiée"""
    try:
        init_excel_file()
        wb = openpyxl.load_workbook(EXCEL_FILE)
        
        if sheet_name not in wb.sheetnames:
            ws = wb.create_sheet(title=sheet_name)
            if sheet_name == "Séances":
                ws.append(["ID", "Date", "Matière", "HeureArrivée", "HeureDepart", "Classe", "IntituléCours", "Centre", "idEnseignant", "NombreEtudiants"])
        else:
            ws = wb[sheet_name]
        
        ws.append(data)
        wb.save(EXCEL_FILE)
        return True
    except Exception as e:
        st.error(f"Erreur lors de la sauvegarde : {e}")
        return False

def read_from_excel(sheet_name):
    """Lit les données d'une feuille Excel"""
    try:
        if os.path.exists(EXCEL_FILE):
            df = pd.read_excel(EXCEL_FILE, sheet_name=sheet_name)
            return df
        else:
            return pd.DataFrame()
    except Exception as e:
        return pd.DataFrame()



def authenticate_teacher(username, password):
    """Authentifie un enseignant"""
    if username in ENSEIGNANTS_DB:
        if ENSEIGNANTS_DB[username]["password"] == password:
            return ENSEIGNANTS_DB[username]
    return None

def login_page():
    """Page de connexion pour les enseignants"""
    st.markdown("""
    <div class="main-header">
        <h1>👨‍🏫 Espace Enseignant</h1>
        <h3>STATO-SPHERE PREPAS</h3>
        <p>Connectez-vous pour accéder à votre espace personnel</p>
    </div>
    """, unsafe_allow_html=True)
    
    st.markdown('<div class="login-container">', unsafe_allow_html=True)
    st.markdown("### 🔐 Connexion Enseignant")
    
    with st.form("login_form"):
        username = st.text_input("👤 Nom d'utilisateur", placeholder="Ex: mbang.pierre")
        password = st.text_input("🔒 Mot de passe", type="password", placeholder="Votre mot de passe")
        
        col1, col2 = st.columns(2)
        with col1:
            login_submitted = st.form_submit_button("🚀 Se connecter", type="primary", use_container_width=True)
        with col2:
            demo_button = st.form_submit_button("👁️ Mode Démo", use_container_width=True)
        
        if login_submitted:
            if username and password:
                teacher_info = authenticate_teacher(username, password)
                if teacher_info:
                    st.session_state.logged_in = True
                    st.session_state.current_teacher = teacher_info
                    st.session_state.teacher_username = username
                    st.success(f"✅ Connexion réussie ! Bienvenue {teacher_info['nom']}")
                    st.rerun()
                else:
                    st.error("❌ Nom d'utilisateur ou mot de passe incorrect")
            else:
                st.warning("⚠️ Veuillez remplir tous les champs")
        
        if demo_button:
            # Mode démo avec Dr. MBANG Pierre
            st.session_state.logged_in = True
            st.session_state.current_teacher = ENSEIGNANTS_DB["mbang.pierre"]
            st.session_state.teacher_username = "mbang.pierre"
            st.success("🎭 Mode démo activé avec Dr. MBANG Pierre")
            st.rerun()
    
    st.markdown('</div>', unsafe_allow_html=True)
    
    # Aide à la connexion
    st.markdown("---")
    st.markdown("### 💡 Comptes de démonstration")
    
    demo_accounts = [
        {"username": "mbang.pierre", "password": "mbang123", "nom": "Dr. MBANG Pierre", "matieres": "Math, Stats"},
        {"username": "njoya.marie", "password": "njoya123", "nom": "Prof. NJOYA Marie", "matieres": "Français, Culture G."},
        {"username": "fomba.jean", "password": "fomba123", "nom": "M. FOMBA Jean", "matieres": "Économie, Compta"},
    ]
    
    for account in demo_accounts:
        st.markdown(f"""
        **{account['nom']}**  
        📧 Username: `{account['username']}` | 🔒 Password: `{account['password']}`  
        📚 Matières: {account['matieres']}
        """)

def main_teacher_interface():
    """Interface principale pour les enseignants connectés"""
    teacher = st.session_state.current_teacher
    username = st.session_state.teacher_username
    
    # En-tête avec informations enseignant
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.markdown(f"""
        <div class="teacher-info">
            <h2>👨‍🏫 {teacher['nom']}</h2>
            <p><strong>Matières :</strong> {', '.join(teacher['matieres'])}</p>
            <p><strong>Centres :</strong> {', '.join(teacher['centres'])}</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        if st.button("🚪 Se déconnecter", type="secondary", use_container_width=True):
            st.session_state.logged_in = False
            st.session_state.current_teacher = None
            st.session_state.teacher_username = None
            st.rerun()
    
    # Statistiques rapides de l'enseignant
    seances_df = read_from_excel("Séances")
    if not seances_df.empty and 'idEnseignant' in seances_df.columns:
        # Filtrer les séances de cet enseignant (ici on utilise le nom comme identifiant)
        teacher_seances = seances_df[seances_df['idEnseignant'] == teacher['nom']]
    else:
        teacher_seances = pd.DataFrame()
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown(f"""
        <div class="stat-card">
            <h3>📚 Séances</h3>
            <h2>{len(teacher_seances)}</h2>
            <p>Dispensées</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        total_heures = 0
        if not teacher_seances.empty and 'HeureArrivée' in teacher_seances.columns:
            # Calcul approximatif des heures (à améliorer)
            total_heures = len(teacher_seances) * 2  # 2h par séance en moyenne
        st.markdown(f"""
        <div class="stat-card">
            <h3>⏱️ Heures</h3>
            <h2>{total_heures}h</h2>
            <p>Enseignées</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        total_etudiants = teacher_seances['NombreEtudiants'].sum() if 'NombreEtudiants' in teacher_seances.columns else 0
        st.markdown(f"""
        <div class="stat-card">
            <h3>👥 Étudiants</h3>
            <h2>{total_etudiants}</h2>
            <p>Touchés</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col4:
        moy_etudiants = int(teacher_seances['NombreEtudiants'].mean()) if len(teacher_seances) > 0 and 'NombreEtudiants' in teacher_seances.columns else 0
        st.markdown(f"""
        <div class="stat-card">
            <h3>📊 Moyenne</h3>
            <h2>{moy_etudiants}</h2>
            <p>Étud./séance</p>
        </div>
        """, unsafe_allow_html=True)
    
    # Interface à onglets
    tab1, tab2, tab3 = st.tabs(["📝 Pointer une Séance", "📚 Mes Séances", "👥 Ajouter un Étudiant"])
    
    # ==================== ONGLET POINTER SÉANCE ====================
    with tab1:
        st.markdown("## 📝 Pointer une Nouvelle Séance")
        
        st.markdown('<div class="form-container">', unsafe_allow_html=True)
        st.markdown("### ➕ Enregistrement de Séance")
        
        with st.form("form_seance"):
            col1, col2 = st.columns(2)
            
            with col1:
                date_seance = st.date_input("📅 Date de la séance *", value=date.today())
                
                # L'enseignant est pré-sélectionné
                nom_enseignant = st.text_input("👨‍🏫 Enseignant", value=teacher['nom'], disabled=True)
                
                cours_dispense = st.selectbox("📚 Cours dispensé *", teacher['matieres'])
                
                intitule_cours = st.text_input(
                    "📝 Intitulé du cours *", 
                    placeholder="Ex: Analyse mathématique - Chapitre 3 : Dérivées"
                )
            
            with col2:
                heure_arrivee = st.time_input("🕐 Heure d'arrivée *", value=time(15, 30))
                
                heure_depart = st.time_input("🕕 Heure de départ *", value=time(17, 30))
                
                centre_seance = st.selectbox("📍 Centre *", teacher['centres'])
                
                nombre_etudiants = st.number_input(
                    "👥 Nombre d'étudiants présents *", 
                    min_value=0, 
                    max_value=100, 
                    value=20
                )
            
            # Classe/Niveau (optionnel)
            classe = st.text_input("🎓 Classe/Niveau", placeholder="Ex: ISE MATH 2025")
            
            submitted_seance = st.form_submit_button("💾 Enregistrer la Séance", type="primary")
            
            if submitted_seance:
                if (date_seance and cours_dispense and intitule_cours and 
                    heure_arrivee and heure_depart and centre_seance and nombre_etudiants >= 0):
                    
                    # Validation des heures
                    if heure_depart <= heure_arrivee:
                        st.markdown("""
                        <div class="error-box">
                            ⚠️ L'heure de départ doit être supérieure à l'heure d'arrivée
                        </div>
                        """, unsafe_allow_html=True)
                    else:
                        # Génération de l'ID
                        seances_df = read_from_excel("Séances")
                        id_seance = len(seances_df) + 1
                        
                        data = [
                            id_seance,
                            date_seance.strftime('%Y-%m-%d'),
                            cours_dispense,
                            heure_arrivee.strftime('%H:%M'),
                            heure_depart.strftime('%H:%M'),
                            classe if classe else "",
                            intitule_cours,
                            centre_seance,
                            teacher['nom'],  # idEnseignant 
                            nombre_etudiants
                        ]
                        
                        if save_to_excel("Séances", data):
                            # Calcul de la durée
                            duree_minutes = (datetime.combine(date.today(), heure_depart) - 
                                           datetime.combine(date.today(), heure_arrivee)).seconds // 60
                            duree_heures = duree_minutes // 60
                            duree_min_restant = duree_minutes % 60
                            
                            st.markdown(f"""
                            <div class="success-box">
                                ✅ <strong>Séance enregistrée avec succès !</strong><br>
                                📚 <strong>Cours :</strong> {cours_dispense}<br>
                                ⏱️ <strong>Durée :</strong> {duree_heures}h{duree_min_restant:02d}<br>
                                👥 <strong>Étudiants :</strong> {nombre_etudiants}
                            </div>
                            """, unsafe_allow_html=True)
                            st.balloons()
                        else:
                            st.markdown("""
                            <div class="error-box">
                                ❌ Erreur lors de l'enregistrement. Veuillez réessayer.
                            </div>
                            """, unsafe_allow_html=True)
                else:
                    st.markdown("""
                    <div class="error-box">
                        ⚠️ Veuillez remplir tous les champs obligatoires (*)
                    </div>
                    """, unsafe_allow_html=True)
        
        st.markdown('</div>', unsafe_allow_html=True)
    
    # ==================== ONGLET MES SÉANCES ====================
    with tab2:
        st.markdown("## 📚 Historique de mes Séances")
        
        if not teacher_seances.empty:
            st.markdown(f"### 📊 {len(teacher_seances)} séance(s) enregistrée(s)")
            
            # Filtres
            col1, col2, col3 = st.columns(3)
            
            with col1:
                if 'Matière' in teacher_seances.columns:
                    matieres_filter = st.multiselect(
                        "📚 Filtrer par matière",
                        options=teacher_seances['Matière'].unique(),
                        default=teacher_seances['Matière'].unique()
                    )
                else:
                    matieres_filter = []
            
            with col2:
                if 'Centre' in teacher_seances.columns:
                    centres_filter = st.multiselect(
                        "📍 Filtrer par centre",
                        options=teacher_seances['Centre'].unique(),
                        default=teacher_seances['Centre'].unique()
                    )
                else:
                    centres_filter = []
            
            with col3:
                # Période
                if 'Date' in teacher_seances.columns:
                    # Convertir en datetime si ce n'est pas déjà fait
                    teacher_seances['Date'] = pd.to_datetime(teacher_seances['Date'])
                    date_min = teacher_seances['Date'].min().date()
                    date_max = teacher_seances['Date'].max().date()
                    
                    periode = st.date_input(
                        "📅 Période",
                        value=(date_min, date_max),
                        min_value=date_min,
                        max_value=date_max
                    )
            
            # Application des filtres
            filtered_seances = teacher_seances.copy()
            
            if matieres_filter:
                filtered_seances = filtered_seances[filtered_seances['Matière'].isin(matieres_filter)]
            
            if centres_filter:
                filtered_seances = filtered_seances[filtered_seances['Centre'].isin(centres_filter)]
            
            # Affichage des séances filtrées
            st.markdown("---")
            
            if not filtered_seances.empty:
                # Tri par date décroissante
                filtered_seances = filtered_seances.sort_values('Date', ascending=False)
                
                for idx, seance in filtered_seances.iterrows():
                    # Calcul de la durée si possible
                    try:
                        heure_a = datetime.strptime(str(seance['HeureArrivée']), '%H:%M').time()
                        heure_d = datetime.strptime(str(seance['HeureDepart']), '%H:%M').time()
                        duree = (datetime.combine(date.today(), heure_d) - 
                               datetime.combine(date.today(), heure_a)).seconds // 60
                        duree_str = f"{duree//60}h{duree%60:02d}"
                    except:
                        duree_str = "N/A"
                    
                    st.markdown(f"""
                    <div class="seance-card">
                        <div style="display: flex; justify-content: space-between; align-items: center;">
                            <div>
                                <h4>📚 {seance['Matière']} - {seance.get('IntituléCours', 'N/A')}</h4>
                                <p><strong>📅 Date:</strong> {seance['Date'].strftime('%d/%m/%Y')} | 
                                   <strong>📍 Centre:</strong> {seance['Centre']} | 
                                   <strong>🎓 Classe:</strong> {seance.get('Classe', 'N/A')}</p>
                                <p><strong>⏰ Horaires:</strong> {seance['HeureArrivée']} - {seance['HeureDepart']} 
                                   ({duree_str}) | 
                                   <strong>👥 Étudiants:</strong> {seance['NombreEtudiants']}</p>
                            </div>
                        </div>
                    </div>
                    """, unsafe_allow_html=True)
                
                # Export des données
                st.markdown("---")
                csv = filtered_seances.to_csv(index=False)
                st.download_button(
                    label="📥 Télécharger mes séances (CSV)",
                    data=csv,
                    file_name=f"seances_{username}_{datetime.now().strftime('%Y%m%d')}.csv",
                    mime="text/csv"
                )
            else:
                st.info("Aucune séance trouvée avec les filtres sélectionnés")
        else:
            st.markdown("""
            <div class="info-box">
                ℹ️ <strong>Aucune séance enregistrée</strong><br>
                Utilisez l'onglet "Pointer une Séance" pour enregistrer votre première séance.
            </div>
            """, unsafe_allow_html=True)
    
    # ==================== ONGLET AJOUTER ÉTUDIANT ====================
    with tab3:
        st.markdown("## 👥 Enregistrer un Nouvel Étudiant")
        
        st.markdown('<div class="form-container">', unsafe_allow_html=True)
        st.markdown("### ➕ Inscription d'un Étudiant")
        
        with st.form("form_etudiant_teacher"):
            col1, col2 = st.columns(2)
            
            with col1:
                nom = st.text_input("Nom *", placeholder="Ex: DUPONT")
                sexe = st.selectbox("Sexe *", SEXE_CHOICES)
                concours1 = st.selectbox("Concours Principal *", CONCOURS_CHOICES)
                etablissement = st.text_input("Établissement", placeholder="Ex: Université de Yaoundé I")
                # Centre limité aux centres de l'enseignant
                centre = st.selectbox("Centre *", teacher['centres'])
            
            with col2:
                prenom = st.text_input("Prénom *", placeholder="Ex: Jean")
                date_arrivee = st.date_input("Date d'arrivée *", value=date.today())
                concours2 = st.selectbox("Concours Secondaire (optionnel)", [""] + CONCOURS_CHOICES)
                concours3 = st.selectbox("Concours Tertiaire (optionnel)", [""] + CONCOURS_CHOICES)
                telephone = st.text_input("Téléphone", placeholder="Ex: +237670123456")
            
            submitted_etudiant = st.form_submit_button("🎓 Enregistrer l'Étudiant", type="primary")
            
            if submitted_etudiant:
                if nom and prenom and sexe and concours1 and centre:
                    matricule = generate_matricule()
                    
                    data = [
                        matricule, nom, prenom, sexe, concours1,
                        concours2 if concours2 else "", concours3 if concours3 else "",
                        telephone, date_arrivee.strftime('%Y-%m-%d'), etablissement, centre
                    ]
                    
                    if save_to_excel("Étudiants", data):
                        st.markdown(f"""
                        <div class="success-box">
                            ✅ <strong>Étudiant enregistré avec succès !</strong><br>
                            Matricule généré: <strong>{matricule}</strong><br>
                            Enregistré par: <strong>{teacher['nom']}</strong>
                        </div>
                        """, unsafe_allow_html=True)
                        st.balloons()
                    else:
                        st.markdown("""
                        <div class="error-box">
                            ❌ Erreur lors de l'enregistrement. Veuillez réessayer.
                        </div>
                        """, unsafe_allow_html=True)
                else:
                    st.markdown("""
                    <div class="error-box">
                        ⚠️ Veuillez remplir tous les champs obligatoires (*)
                    </div>
                    """, unsafe_allow_html=True)
        
        st.markdown('</div>', unsafe_allow_html=True)

def main():
    # Initialisation des variables de session
    if "logged_in" not in st.session_state:
        st.session_state.logged_in = False
    
    if "current_teacher" not in st.session_state:
        st.session_state.current_teacher = None
    
    if "teacher_username" not in st.session_state:
        st.session_state.teacher_username = None
    
    # Affichage conditionnel selon l'état de connexion
    if not st.session_state.logged_in:
        login_page()
    else:
        main_teacher_interface()

if __name__ == "__main__":
    main()